<a href="https://colab.research.google.com/github/rkb-1/mrca/blob/master/SCARA_Kinematics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Forward Kinematic Analysis of SCARA robot

![picture](https://drive.google.com/uc?export=view&id=1A-A9vGlx-TqWX7HH776gvCHnsl50wX9v)
Source: Modern Robotics, 2017 

**SCARA Pick and Place Robot Arm**
The SCARA is a type of industrial robot. The acronym stands for Selective Compliance Assembly Robot Arm. It is of type RRRP with three revolute joints and one prismatic joint connecting the links in series respectively.

In [ ]:
# Import required libraries
import numpy as np
from sympy import Matrix, symbols, pretty_print, Identity, cos, sin, zeros, simplify

## Spatial Representation of Screw axes


In [ ]:
l1, l2, l0 = symbols("l1 l2 l0", real=1)  # link lenghts

In [ ]:
s1_hat = Matrix([0,0,1])
s1_o = Matrix([0,0,0])
display(s1_hat,s1_o)

In [ ]:
s2_hat = Matrix([0,0,1])
s2_o = Matrix([0,l1,0])
display(s2_hat,s2_o)

In [ ]:
s3_hat = Matrix([0,0,1])
s3_o = Matrix([0,l1+l2,0])
display(s3_hat,s3_o)

In [ ]:
## Prismatic joint
s4_hat = Matrix([0,0,1])
display(s4_hat)

In [ ]:
S1 = Matrix([s1_hat, s1_o.cross(s1_hat)])
S2 = Matrix([s2_hat, s2_o.cross(s2_hat)])
S3 = Matrix([s3_hat, s3_o.cross(s3_hat)])
S4 = Matrix([Matrix([0,0,0]), s4_hat])
print("S1:")
display(S1)
print("S2:")
display(S2)
print("S3:")
display(S3)
print("S4:")
display(S4)
S = S1.row_join(S2).row_join(S3).row_join(S4)
print("Screw axes (arranged column wise for the joints) :\n")
display(S)

## Reference configuration of the end effector at zero position
A general configuration is given by a rotation matrix $\mathbf{R}$ and position vector $\mathbf{p}$ as follows.
\begin{equation}
        \mathbf{T} = \begin{bmatrix}
                \mathbf{R} & \textbf{p} \\ \mathbf{0} & 1
            \end{bmatrix}
    \end{equation}



In [ ]:
p_0_ee = Matrix([ 0, l1+l2 , l0 ])
R_0_ee = Matrix(Identity(3))
T_0_ee = R_0_ee.row_join(p_0_ee).col_join(Matrix([0, 0, 0, 1]).T)
print("Zero configuration of end-effector pose: \n")
display(T_0_ee)

## Product of Exponential definition

Let $ \mathbf S = (\boldsymbol \xi, \boldsymbol \eta) $ denote the screw coordinates. 
The matrix exponential is defined as $\exp: [\mathbf S] \theta \in se(3) \to \mathbf T \in SE(3)$. 
 
If $||{\boldsymbol \xi}|| = 1$ then for any distance $\theta \in \mathbb{R}$ traveled along the screw axis or any angle $\theta \in \mathbb{R}$ rotated about the screw axis,
\begin{equation}
 \mathbf T = 
  \exp{[\mathbf S] \theta} 
  = \left[
\begin{matrix}
\exp{[\boldsymbol \xi] \theta} & (\mathbf I \theta + (1-\cos \theta)[\boldsymbol \xi] + (\theta - \sin \theta)[\boldsymbol \xi]^2) \boldsymbol \eta\\
\mathbf 0 & 1
\end{matrix}
 \right] \in SE(3)
 \end{equation}
where, 
\begin{equation}
\mathbf R = \exp{[\boldsymbol \xi] \theta} = \mathbf I + \sin \theta[\boldsymbol \xi] + (1 - \cos \theta)[\boldsymbol \xi]^2 \in SO(3) 
\end{equation}
and
\begin{equation}
\mathbf p = (\mathbf I \theta + (1-\cos \theta)[\boldsymbol \xi] + (\theta - \sin \theta)[\boldsymbol \xi]^2) \boldsymbol \eta \in \mathbb R^3 
\end{equation}
If $||{\boldsymbol \xi}|| = 0$ and $||{\boldsymbol \eta}|| = 1$ (prismatic joints), then
 \begin{equation}
  \mathbf T = 
  \exp{[\mathbf S] \theta} 
  = \left[
\begin{matrix}
\mathbf I &  \boldsymbol \eta \theta\\
\mathbf 0 & 1
\end{matrix}
 \right] \in SE(3) 
 \end{equation}




### The product of exponential (POE) formula is given by
\begin{equation}
  {}^0\mathbf T_n(\mathbf q) = \exp([\mathbf S_1]q_1) \exp([\mathbf S_2]q_2) \ldots \exp([\mathbf S_n]q_n) {}^0\mathbf T_n(\mathbf 0) 
  \label{eqn_POE}
 \end{equation}


In [ ]:
def SE3Exp(XX, t):
        """compute exponential mapping for SE(3).
        """
        X = XX.T
        xi = Matrix(X[0:3])
        eta = Matrix(X[3:6])
        xihat = Matrix([[0, -xi[2], xi[1]],
                        [xi[2], 0, -xi[0]],
                        [-xi[1], xi[0], 0]])
        R = Matrix(Identity(3)) + sin(t)*xihat + (1-cos(t))*(xihat*xihat)
        if xi == zeros(3, 1):
            p = eta * t
        else:
            p = Matrix(Identity(3)*t + (1-cos(t))*xihat + (t-sin(t))*(xihat*xihat))*eta
        T = R.row_join(p).col_join(Matrix([0, 0, 0, 1]).T)
        return T
print("Matrix exponential for screw axis 3 : \n")
display(SE3Exp(S3,symbols("x")))#example

## Forward Kinematics Using POE formula
### Define the generalised coordinates for the system, $q_1 \dots q_6$

In [ ]:
q1, q2, q3, q4 = symbols("q1 q2 q3 q4", real=1)  # joint positions
T_0_q = simplify(SE3Exp(S1,q1) * SE3Exp(S2,q2) * SE3Exp(S3,q3) * SE3Exp(S4,q4) * T_0_ee)
print("Forward kinematics : \n")
display(T_0_q)

### Substitute with numerical values
q1 = 0
q2 = np.pi/2
q3 = -np.pi/2
q4 = 1
l0 = l1 = l2 = 1

In [ ]:
fkin= T_0_q.evalf(4, subs={q1:0, q2:np.pi/2, q3:-np.pi/2, q4:1,
                          l0: 1, l1: 1, l2: 1})
print("Forward kinematics for the input joint angles: \n")
display(fkin)

# Using Modern Robotics toolbox

In [ ]:
# Import required libraries
!pip install modern_robotics
import modern_robotics as mr

In [ ]:
Slist = np.asarray(S.subs({l0: 1, l1: 1, l2: 1})).astype(np.float64)
T_zero = np.asarray(T_0_ee.subs({l0: 1, l1: 1, l2: 1})).astype(np.float64)
q=np.asarray([0, np.pi/2, -np.pi/2, 1])
print("List of screw axes: \n", Slist)
print("Zero configuration pose of end effector frame: \n", T_zero)
print("Input angles: \n", q)

In [ ]:
fkin_mr = mr.FKinSpace(T_zero,Slist,q)
display(fkin_mr)

# Inverse Kinematics of SCARA

In [ ]:
from sympy import Eq, solve
# T_q = T_0_q.subs({l0: 1, l1: 1, l2: 1})
T_q = T_0_q
T_q


In [ ]:
x, y, z, phi = symbols("x y z phi", real=1)

In [ ]:
x_ = T_q[0,3]
y_ = T_q[1,3]
z_ = T_q[2,3]

## Solve for $q_4$

In [ ]:
q4_sol = solve(Eq(z-z_),q4)
q4_sol[0]

## Solve for $q_2$

In [ ]:
f1 = (x**2 +  y**2)
f2 = (x_**2 +  y_**2)
eq1 = simplify(Eq(f1-f2))
eq1

In [ ]:
sol = solve(eq1,q2)
sol[1]

## Solve for $q_1$

In [ ]:
f3 = simplify(-x*sin(q1) + y* cos(q1))
f3

In [ ]:
f4 = simplify(x_*(-sin(q1)) + y_*(cos(q1)))
simplify(f4)

In [ ]:
eq2 = simplify(Eq(f3-f4))
eq2

In [ ]:
f5 = simplify(x*cos(q1) + y*sin(q1))
f5

In [ ]:
f6 = simplify(x_*cos(q1) + y_*sin(q1))
f6

In [ ]:
eq3 = (Eq(f5/f3 - f6/f4))
eq3

## Sovle for $q_3$

In [ ]:
q3_sol = solve(Eq(phi-(q1+q2+q3)),q3)
q3_sol[0]